In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)
from datetime import timedelta
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
data=pd.read_csv('icu_data_12hour_max,min,mean.csv')
data

/var/folders/n0/1sprhnln7nl_hy_j6sj6xzr00000gn/T/ipykernel_85558/540397051.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('icu_data_12hour_max,min,mean.csv')


,STAY_ID,time,max_RR,max_BT,max_DBP,max_mean BP,max_SBP,max_HR,max_E(eye)/V(verbal)/M(motor),max_SpO2,max_FIO2,min_RR,min_BT,min_DBP,min_mean BP,min_SBP,min_HR,min_E(eye)/V(verbal)/M(motor),min_SpO2,min_FIO2,mean_RR,mean_BT,mean_DBP,mean_mean BP,mean_SBP,mean_HR,mean_E(eye)/V(verbal)/M(motor),mean_SpO2,mean_FIO2,"max_Glucose(BST,POCT)","max_Sodium(Na),응급","max_BUN,응급","max_Chloride(Cl),응급","max_Potassium(K),응급","max_GFR by MDRD,응급","max_Creatinine,응급","max_Glucose,WB",max_Standard bicarbonate,max_O2Hb,max_MetHb,"max_Potassium,WB",max_Chloride,max_Sodium,max_Actual base excess,"max_Chloride,WB",max_Standard base excess,max_pO2,max_HHb,"max_Ionized calcium,WB",max_Oxygen saturation,"max_Sodium,WB",max_pCO2,max_Total Hb,"max_Lactate,WB",max_Potassium,"max_pH(ER,ICU,CCU,OR)",max_HCO3- conc.,max_COHb,max_Total CO2 conc.,"min_Glucose(BST,POCT)","min_Sodium(Na),응급","min_BUN,응급","min_Chloride(Cl),응급","min_Potassium(K),응급","min_GFR by MDRD,응급","min_Creatinine,응급","min_Glucose,WB",min_Standard bicarbonate,min_O2Hb,min_MetHb,"min_Potassium,WB",min_Chloride,min_Sodium,min_Actual base excess,"min_Chloride,WB",min_Standard base excess,min_pO2,min_HHb,"min_Ionized calcium,WB",min_Oxygen saturation,"min_Sodium,WB",min_pCO2,min_Total Hb,"min_Lactate,WB",min_Potassium,"min_pH(ER,ICU,CCU,OR)",min_HCO3- conc.,min_COHb,min_Total CO2 conc.,"mean_Glucose(BST,POCT)","mean_Sodium(Na),응급","mean_BUN,응급","mean_Chloride(Cl),응급","mean_Potassium(K),응급","mean_GFR by MDRD,응급","mean_Creatinine,응급","mean_Glucose,WB",mean_Standard bicarbonate,mean_O2Hb,mean_MetHb,"mean_Potassium,WB",mean_Chloride,mean_Sodium,mean_Actual base excess,"mean_Chloride,WB",mean_Standard base excess,mean_pO2,mean_HHb,"mean_Ionized calcium,WB",mean_Oxygen saturation,"mean_Sodium,WB",mean_pCO2,mean_Total Hb,"mean_Lactate,WB",mean_Potassium,"mean_pH(ER,ICU,CCU,OR)",mean_HCO3- conc.,mean_COHb,mean_Total CO2 conc.,inpute_max_RR,inpute_max_BT,inpute_max_DBP,inpute_max_mean BP,inpute_max_SBP,inpute_max_HR,inpute_max_E(eye)/V(verbal)/M(motor),inpute_max_SpO2,inpute_max_FIO2,inpute_min_RR,inpute_min_BT,inpute_min_DBP,inpute_min_mean BP,inpute_min_SBP,inpute_min_HR,inpute_min_E(eye)/V(verbal)/M(motor),inpute_min_SpO2,inpute_min_FIO2,inpute_mean_RR,inpute_mean_BT,inpute_mean_DBP,inpute_mean_mean BP,inpute_mean_SBP,inpute_mean_HR,inpute_mean_E(eye)/V(verbal)/M(motor),inpute_mean_SpO2,inpute_mean_FIO2,"inpute_max_Glucose(BST,POCT)","inpute_max_Sodium(Na),응급","inpute_max_BUN,응급","inpute_max_Chloride(Cl),응급","inpute_max_Potassium(K),응급","inpute_max_GFR by MDRD,응급","inpute_max_Creatinine,응급","inpute_max_Glucose,WB",inpute_max_Standard bicarbonate,inpute_max_O2Hb,inpute_max_MetHb,"inpute_max_Potassium,WB",inpute_max_Chloride,inpute_max_Sodium,inpute_max_Actual base excess,"inpute_max_Chloride,WB",inpute_max_Standard base excess,inpute_max_pO2,inpute_max_HHb,"inpute_max_Ionized calcium,WB",inpute_max_Oxygen saturation,"inpute_max_Sodium,WB",inpute_max_pCO2,inpute_max_Total Hb,"inpute_max_Lactate,WB",inpute_max_Potassium,"inpute_max_pH(ER,ICU,CCU,OR)",inpute_max_HCO3- conc.,inpute_max_COHb,inpute_max_Total CO2 conc.,"inpute_min_Glucose(BST,POCT)","inpute_min_Sodium(Na),응급","inpute_min_BUN,응급","inpute_min_Chloride(Cl),응급","inpute_min_Potassium(K),응급","inpute_min_GFR by MDRD,응급","inpute_min_Creatinine,응급","inpute_min_Glucose,WB",inpute_min_Standard bicarbonate,inpute_min_O2Hb,inpute_min_MetHb,"inpute_min_Potassium,WB",inpute_min_Chloride,inpute_min_Sodium,inpute_min_Actual base excess,"inpute_min_Chloride,WB",inpute_min_Standard base excess,inpute_min_pO2,inpute_min_HHb,"inpute_min_Ionized calcium,WB",inpute_min_Oxygen saturation,"inpute_min_Sodium,WB",inpute_min_pCO2,inpute_min_Total Hb,"inpute_min_Lactate,WB",inpute_min_Potassium,"inpute_min_pH(ER,ICU,CCU,OR)",inpute_min_HCO3- conc.,inpute_min_COHb,inpute_min_Total CO2 conc.,"inpute_mean_Glucose(BST,POCT)","inpute_mean_Sodium(Na),응급","inpute_mean_BUN,응급","inpute_mean_Chloride(Cl),응급","inpute_mean_Potassium(K),응급","inpute_mean_GFR by MDRD,응급","inpute_mean

In [3]:
len(data[data['real_dead']==1]['STAY_ID'].unique())

35

In [4]:
35/2

17.5

# lstm_ED 데이터 train, val, test, val test

In [6]:
data['STAY_ID'] = data['STAY_ID'].astype(str)

# 이상한(사망) STAY_ID 추출
hadm_ids_ab_label = data[data['real_dead'] == 1]['STAY_ID'].unique()
print(f"abnormal_hadm_id_num: {len(hadm_ids_ab_label)}")

# 검증 및 테스트 용도로 이상한 STAY_ID에서 무작위 선택
val_ab_test_id = np.random.choice(hadm_ids_ab_label, size=17, replace=False)
print(f"ab_val_test_id: {len(val_ab_test_id)}")

# 나머지 STAY_ID
ab_test_id = np.setdiff1d(hadm_ids_ab_label, val_ab_test_id)
print(f"ab_test_id: {len(ab_test_id)}")

# 교집합 검사
set(val_ab_test_id) & set(ab_test_id)

abnormal_hadm_id_num: 35
ab_val_test_id: 17
ab_test_id: 18


set()

In [27]:
len(hadm_id_live_label)

605

In [8]:
hadm_id_live_label = data[data['real_dead'] == 0]['STAY_ID'].unique()
print(f"normal_hadm_id_num:{len(hadm_id_live_label)}")

normal_test_id = np.random.choice(hadm_id_live_label, size=35, replace=False)
print(f"normal_test_id:{len(normal_test_id)}")

val_nor_test_id = np.random.choice(normal_test_id, size=17, replace=False)
print(f"nor_val_test_id:{len(val_nor_test_id)}")

nor_test_id = np.setdiff1d(normal_test_id, val_nor_test_id)
print(f"nor_test_id:{len(nor_test_id)}")

real_normal = np.setdiff1d(hadm_id_live_label, normal_test_id)
print(f"real_normal:{len(real_normal)}")

train_id = np.random.choice(real_normal, size=round(len(real_normal)*0.9), replace=False)
print(f"train_id:{len(train_id)}")

val_id = np.setdiff1d(real_normal, train_id)
print(f"val_id:{len(val_id)}")

set(train_id)&set(val_id)&set(val_nor_test_id)&set(nor_test_id)

normal_hadm_id_num:605
normal_test_id:35
nor_val_test_id:17
nor_test_id:18
real_normal:570
train_id:513
val_id:57


set()

In [9]:
print(f"train_id:{len(train_id)}")
print(f"val_id:{len(val_id)}")

val_test_id=set(val_nor_test_id)|set(val_ab_test_id)
test_id=set(nor_test_id)|set(ab_test_id)

print(f"val_test_id:{len(val_test_id)}")
print(f"test_id:{len(test_id)}")

train_id:513
val_id:57
val_test_id:34
test_id:36


In [10]:
columns_to_drop = ['real_dead']
result_df_4 = data.drop(columns=columns_to_drop)

In [11]:
result_df_4

,STAY_ID,time,max_RR,max_BT,max_DBP,max_mean BP,max_SBP,max_HR,max_E(eye)/V(verbal)/M(motor),max_SpO2,max_FIO2,min_RR,min_BT,min_DBP,min_mean BP,min_SBP,min_HR,min_E(eye)/V(verbal)/M(motor),min_SpO2,min_FIO2,mean_RR,mean_BT,mean_DBP,mean_mean BP,mean_SBP,mean_HR,mean_E(eye)/V(verbal)/M(motor),mean_SpO2,mean_FIO2,"max_Glucose(BST,POCT)","max_Sodium(Na),응급","max_BUN,응급","max_Chloride(Cl),응급","max_Potassium(K),응급","max_GFR by MDRD,응급","max_Creatinine,응급","max_Glucose,WB",max_Standard bicarbonate,max_O2Hb,max_MetHb,"max_Potassium,WB",max_Chloride,max_Sodium,max_Actual base excess,"max_Chloride,WB",max_Standard base excess,max_pO2,max_HHb,"max_Ionized calcium,WB",max_Oxygen saturation,"max_Sodium,WB",max_pCO2,max_Total Hb,"max_Lactate,WB",max_Potassium,"max_pH(ER,ICU,CCU,OR)",max_HCO3- conc.,max_COHb,max_Total CO2 conc.,"min_Glucose(BST,POCT)","min_Sodium(Na),응급","min_BUN,응급","min_Chloride(Cl),응급","min_Potassium(K),응급","min_GFR by MDRD,응급","min_Creatinine,응급","min_Glucose,WB",min_Standard bicarbonate,min_O2Hb,min_MetHb,"min_Potassium,WB",min_Chloride,min_Sodium,min_Actual base excess,"min_Chloride,WB",min_Standard base excess,min_pO2,min_HHb,"min_Ionized calcium,WB",min_Oxygen saturation,"min_Sodium,WB",min_pCO2,min_Total Hb,"min_Lactate,WB",min_Potassium,"min_pH(ER,ICU,CCU,OR)",min_HCO3- conc.,min_COHb,min_Total CO2 conc.,"mean_Glucose(BST,POCT)","mean_Sodium(Na),응급","mean_BUN,응급","mean_Chloride(Cl),응급","mean_Potassium(K),응급","mean_GFR by MDRD,응급","mean_Creatinine,응급","mean_Glucose,WB",mean_Standard bicarbonate,mean_O2Hb,mean_MetHb,"mean_Potassium,WB",mean_Chloride,mean_Sodium,mean_Actual base excess,"mean_Chloride,WB",mean_Standard base excess,mean_pO2,mean_HHb,"mean_Ionized calcium,WB",mean_Oxygen saturation,"mean_Sodium,WB",mean_pCO2,mean_Total Hb,"mean_Lactate,WB",mean_Potassium,"mean_pH(ER,ICU,CCU,OR)",mean_HCO3- conc.,mean_COHb,mean_Total CO2 conc.,inpute_max_RR,inpute_max_BT,inpute_max_DBP,inpute_max_mean BP,inpute_max_SBP,inpute_max_HR,inpute_max_E(eye)/V(verbal)/M(motor),inpute_max_SpO2,inpute_max_FIO2,inpute_min_RR,inpute_min_BT,inpute_min_DBP,inpute_min_mean BP,inpute_min_SBP,inpute_min_HR,inpute_min_E(eye)/V(verbal)/M(motor),inpute_min_SpO2,inpute_min_FIO2,inpute_mean_RR,inpute_mean_BT,inpute_mean_DBP,inpute_mean_mean BP,inpute_mean_SBP,inpute_mean_HR,inpute_mean_E(eye)/V(verbal)/M(motor),inpute_mean_SpO2,inpute_mean_FIO2,"inpute_max_Glucose(BST,POCT)","inpute_max_Sodium(Na),응급","inpute_max_BUN,응급","inpute_max_Chloride(Cl),응급","inpute_max_Potassium(K),응급","inpute_max_GFR by MDRD,응급","inpute_max_Creatinine,응급","inpute_max_Glucose,WB",inpute_max_Standard bicarbonate,inpute_max_O2Hb,inpute_max_MetHb,"inpute_max_Potassium,WB",inpute_max_Chloride,inpute_max_Sodium,inpute_max_Actual base excess,"inpute_max_Chloride,WB",inpute_max_Standard base excess,inpute_max_pO2,inpute_max_HHb,"inpute_max_Ionized calcium,WB",inpute_max_Oxygen saturation,"inpute_max_Sodium,WB",inpute_max_pCO2,inpute_max_Total Hb,"inpute_max_Lactate,WB",inpute_max_Potassium,"inpute_max_pH(ER,ICU,CCU,OR)",inpute_max_HCO3- conc.,inpute_max_COHb,inpute_max_Total CO2 conc.,"inpute_min_Glucose(BST,POCT)","inpute_min_Sodium(Na),응급","inpute_min_BUN,응급","inpute_min_Chloride(Cl),응급","inpute_min_Potassium(K),응급","inpute_min_GFR by MDRD,응급","inpute_min_Creatinine,응급","inpute_min_Glucose,WB",inpute_min_Standard bicarbonate,inpute_min_O2Hb,inpute_min_MetHb,"inpute_min_Potassium,WB",inpute_min_Chloride,inpute_min_Sodium,inpute_min_Actual base excess,"inpute_min_Chloride,WB",inpute_min_Standard base excess,inpute_min_pO2,inpute_min_HHb,"inpute_min_Ionized calcium,WB",inpute_min_Oxygen saturation,"inpute_min_Sodium,WB",inpute_min_pCO2,inpute_min_Total Hb,"inpute_min_Lactate,WB",inpute_min_Potassium,"inpute_min_pH(ER,ICU,CCU,OR)",inpute_min_HCO3- conc.,inpute_min_COHb,inpute_min_Total CO2 conc.,"inpute_mean_Glucose(BST,POCT)","inpute_mean_Sodium(Na),응급","inpute_mean_BUN,응급","inpute_mean_Chloride(Cl),응급","inpute_mean_Potassium(K),응급","inpute_mean_GFR by MDRD,응급","inpute_mean

In [48]:
#columns_to_drop = ['result']
#result_df_2 = result_df_1.drop(columns=columns_to_drop)

In [13]:
train_df=result_df_4[result_df_4['STAY_ID'].isin(train_id)]

val_df=result_df_4[result_df_4['STAY_ID'].isin(val_id)]

val_test_df=result_df_4[result_df_4['STAY_ID'].isin(val_test_id)]

test_df=result_df_4[result_df_4['STAY_ID'].isin(test_id)]

In [14]:
train_df.reset_index(inplace=True,drop=True)
val_df.reset_index(inplace=True,drop=True)
val_test_df.reset_index(inplace=True,drop=True)
test_df.reset_index(inplace=True,drop=True)

# scaler

In [12]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler1 = MinMaxScaler()
#scaler2 = StandardScaler()
#columns_to_scale = train_x.columns[2:].tolist()
one_hot = train_df.columns[0:2].tolist()#+train_df.columns[20:58].tolist()+train_df.columns[59:105].tolist()+train_df.columns[111:].tolist()
var = [col for col in train_df.columns if col not in one_hot]

train_x = train_df.copy()  
train_x[var] = scaler1.fit_transform(train_x[var]) #scaler1 할땐 바꿔줘야함

val_x = val_df.copy()
val_test_x = val_test_df.copy()
test_x = test_df.copy()

val_x[var] = scaler1.transform(val_x[var])
val_test_x[var] = scaler1.transform(val_test_x[var])
test_x[var] = scaler1.transform(test_x[var])

In [13]:
train_df

,hadm_id,time,curr_service_CMED,curr_service_CSURG,curr_service_DENT,curr_service_ENT,curr_service_EYE,curr_service_GU,curr_service_GYN,curr_service_MED,curr_service_NMED,curr_service_NSURG,curr_service_OBS,curr_service_OMED,curr_service_ORTHO,curr_service_PSURG,curr_service_PSYCH,curr_service_SURG,curr_service_TRAUM,curr_service_TSURG,curr_service_VSURG,drug_acetaminophen,drug_aspirin,drug_bisacodyl,drug_docusatesodium,drug_furosemide,drug_heparin,drug_hydromorphone(dilaudid),drug_insulin,drug_lorazepam,drug_magnesiumsulfate,drug_metoprololtartrate,drug_morphinesulfate,drug_ondansetron,drug_oxycodone(immediaterelease),drug_pantoprazole,drug_potassiumchloride,drug_senna,drug_sodiumchloride0.9%flush,drug_vancomycin,abnormal,max_Basophils,max_Eosinophils,max_HCT,max_hemoglobin,max_Lymphocytes,max_MCH,max_MCHC,max_MCV,max_Monocytes,max_Neutrophils,max_Platelet,max_RDW,max_Red Blood Cells,max_WBC,max_ALT,max_Alb,max_Alk_Phos,max_AG,max_AST,max_bicarb,max_T_Bil,"max_Calcium, Total",max_Cl,max_SCr,max_Gl,max_Mg,max_PHOS,max_Potassium,max_Na,max_BUN,max_INR,max_PT,max_PTT,max_fibrinogen,max_thrombin,max_ldh,max_nt_probnp,max_baseexcess,max_paco2,max_PH,max_PaO2,max_ck_mb,max_tropt,max_CK,max_lactate,max_crp,max_D_Bil,max_fio2,max_d_dimer,max_ggt,max_o2sat,min_Basophils,min_Eosinophils,min_HCT,min_hemoglobin,min_Lymphocytes,min_MCH,min_MCHC,min_MCV,min_Monocytes,min_Neutrophils,min_Platelet,min_RDW,min_Red Blood Cells,min_WBC,min_ALT,min_Alb,min_Alk_Phos,min_AG,min_AST,min_bicarb,min_T_Bil,"min_Calcium, Total",min_Cl,min_SCr,min_Gl,min_Mg,min_PHOS,min_Potassium,min_Na,min_BUN,min_INR,min_PT,min_PTT,min_fibrinogen,min_thrombin,min_ldh,min_nt_probnp,min_baseexcess,min_paco2,min_PH,min_PaO2,min_ck_mb,min_tropt,min_CK,min_lactate,min_crp,min_D_Bil,min_fio2,min_d_dimer,min_ggt,min_o2sat,mean_Basophils,mean_Eosinophils,mean_HCT,mean_hemoglobin,mean_Lymphocytes,mean_MCH,mean_MCHC,mean_MCV,mean_Monocytes,mean_Neutrophils,mean_Platelet,mean_RDW,mean_Red Blood Cells,mean_WBC,mean_ALT,mean_Alb,mean_Alk_Phos,mean_AG,mean_AST,mean_bicarb,mean_T_Bil,"mean_Calcium, Total",mean_Cl,mean_SCr,mean_Gl,mean_Mg,mean_PHOS,mean_Potassium,mean_Na,mean_BUN,mean_INR,mean_PT,mean_PTT,mean_fibrinogen,mean_thrombin,mean_ldh,mean_nt_probnp,mean_baseexcess,mean_paco2,mean_PH,mean_PaO2,mean_ck_mb,mean_tropt,mean_CK,mean_lactate,mean_crp,mean_D_Bil,mean_fio2,mean_d_dimer,mean_ggt,mean_o2sat,inpute_max_Eosinophils,inpute_max_HCT,inpute_max_hemoglobin,inpute_max_Lymphocytes,inpute_max_MCH,inpute_max_MCHC,inpute_max_MCV,inpute_max_Monocytes,inpute_max_Neutrophils,inpute_max_Platelet,inpute_max_RDW,inpute_max_Red Blood Cells,inpute_max_WBC,inpute_max_ALT,inpute_max_Alb,inpute_max_Alk_Phos,inpute_max_AG,inpute_max_AST,inpute_max_bicarb,inpute_max_T_Bil,"inpute_max_Calcium, Total",inpute_max_Cl,inpute_max_SCr,inpute_max_Gl,inpute_max_Mg,inpute_max_PHOS,inpute_max_Potassium,inpute_max_Na,inpute_max_BUN,inpute_max_INR,inpute_max_PT,inpute_max_PTT,inpute_max_fibrinogen,inpute_max_thrombin,inpute_max_ldh,inpute_max_nt_probnp,inpute_max_baseexcess,inpute_max_paco2,inpute_max_PH,inpute_max_PaO2,inpute_max_ck_mb,inpute_max_tropt,inpute_max_CK,inpute_max_lactate,inpute_max_crp,inpute_max_D_Bil,inpute_max_fio2,inpute_max_d_dimer,inpute_max_ggt,inpute_max_o2sat,inpute_min_Basophils,inpute_min_Eosinophils,inpute_min_HCT,inpute_min_hemoglobin,inpute_min_Lymphocytes,inpute_min_MCH,inpute_min_MCHC,inpute_min_MCV,inpute_min_Monocytes,inpute_min_Neutrophils,inpute_min_Platelet,inpute_min_RDW,inpute_min_Red Blood Cells,inpute_min_WBC,inpute_min_ALT,inpute_min_Alb,inpute_min_Alk_Phos,inpute_min_AG,inpute_min_AST,inpute_min_bicarb,inpute_min_T_Bil,"inpute_min_Calcium, Total",inpute_min_Cl,inpute_min_SCr,inpute_min_Gl,inpute_min_Mg,inpute_min_PHOS,inpute_min_Potassium,inpute_min_Na,inpute_min_BUN,inpute_min_INR,inpute_min_PT,inpute_min_PTT,inpute_min_fibrinogen,inpute_min_thrombin,inpute_min_ldh,inpute_min_nt_probnp,inpute_min_baseexcess,inpute_min_paco2,inpute_min_PH,inpute_min_PaO2,inpute_mi

In [39]:
train_df.columns[0:40].tolist()+train_df.columns[194:346].tolist()+train_df.columns[347:].tolist()

['hadm_id',
 'time',
 'curr_service_CMED',
 'curr_service_CSURG',
 'curr_service_DENT',
 'curr_service_ENT',
 'curr_service_EYE',
 'curr_service_GU',
 'curr_service_GYN',
 'curr_service_MED',
 'curr_service_NMED',
 'curr_service_NSURG',
 'curr_service_OBS',
 'curr_service_OMED',
 'curr_service_ORTHO',
 'curr_service_PSURG',
 'curr_service_PSYCH',
 'curr_service_SURG',
 'curr_service_TRAUM',
 'curr_service_TSURG',
 'curr_service_VSURG',
 'drug_acetaminophen',
 'drug_aspirin',
 'drug_bisacodyl',
 'drug_docusatesodium',
 'drug_furosemide',
 'drug_heparin',
 'drug_hydromorphone(dilaudid)',
 'drug_insulin',
 'drug_lorazepam',
 'drug_magnesiumsulfate',
 'drug_metoprololtartrate',
 'drug_morphinesulfate',
 'drug_ondansetron',
 'drug_oxycodone(immediaterelease)',
 'drug_pantoprazole',
 'drug_potassiumchloride',
 'drug_senna',
 'drug_sodiumchloride0.9%flush',
 'drug_vancomycin',
 'inpute_max_Eosinophils',
 'inpute_max_HCT',
 'inpute_max_hemoglobin',
 'inpute_max_Lymphocytes',
 'inpute_max_MCH',

In [40]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#scaler1 = MinMaxScaler()
scaler2 = StandardScaler()
#columns_to_scale = train_x.columns[2:].tolist()
one_hot = train_df.columns[0:40].tolist()+train_df.columns[194:346].tolist()+train_df.columns[347:].tolist()
var = [col for col in train_df.columns if col not in one_hot]

train_x = train_df.copy()  
train_x[var] = scaler2.fit_transform(train_x[var]) #scaler1 할땐 바꿔줘야함

val_x = val_df.copy()
val_test_x = val_test_df.copy()
test_x = test_df.copy()

val_x[var] = scaler2.transform(val_x[var])
val_test_x[var] = scaler2.transform(val_test_x[var])
test_x[var] = scaler2.transform(test_x[var])

In [16]:
train_df['STAY_ID'].value_counts()

STAY_ID
20123256.0                  59
006506000233992132091102    59
166703.0                    54
006506000235322099041002    53
167911.0                    52
                            ..
166939.0                     1
166950.0                     1
166982.0                     1
166993.0                     1
167253.0                     1
Name: count, Length: 513, dtype: int64

In [17]:
def pad_sequences(dataframe, padding_label, sequence_length=59):
    hadm_ids = dataframe['STAY_ID'].unique()
    num_vars = dataframe.shape[1]
    padded_data = []
    
    for hadm_id in tqdm(hadm_ids):
        group_data = dataframe[dataframe['STAY_ID'] == hadm_id].iloc[:, :].values
        num_rows = group_data.shape[0]
        
        if num_rows < sequence_length:
            padding_needed = sequence_length - num_rows
            padding_array = np.full((padding_needed, num_vars), padding_label)
            padded_group_data = np.concatenate((group_data, padding_array))
        else:
            padded_group_data = group_data
        
        padded_data.append(padded_group_data)
    
    padded_array = np.concatenate(padded_data)
    padded_df = pd.DataFrame(padded_array, columns=dataframe.columns.tolist())
    padded_df['STAY_ID'].replace(0, np.nan, inplace=True)
    padded_df['STAY_ID'].fillna(method='ffill', inplace=True)
    
    return padded_df

In [18]:
train_x_padded = pad_sequences(train_df, 0) #train_df, train_x
val_x_padded = pad_sequences(val_df, 0)
val_test_x_padded = pad_sequences(val_test_df, 0)
test_x_padded = pad_sequences(test_df, 0)

100%|██████████| 513/513 [00:00<00:00, 1616.40it/s]
/var/folders/n0/1sprhnln7nl_hy_j6sj6xzr00000gn/T/ipykernel_85558/2904734669.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  padded_df['STAY_ID'].replace(0, np.nan, inplace=True)
/var/folders/n0/1sprhnln7nl_hy_j6sj6xzr00000gn/T/ipykernel_85558/2904734669.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because 

# test 데이터 전처리

In [19]:
dfs=[val_test_x_padded, test_x_padded]

In [20]:
val_test_x_padded.shape[1]

268

In [21]:
val_test_x_padded

,STAY_ID,time,max_RR,max_BT,max_DBP,max_mean BP,max_SBP,max_HR,max_E(eye)/V(verbal)/M(motor),max_SpO2,max_FIO2,min_RR,min_BT,min_DBP,min_mean BP,min_SBP,min_HR,min_E(eye)/V(verbal)/M(motor),min_SpO2,min_FIO2,mean_RR,mean_BT,mean_DBP,mean_mean BP,mean_SBP,mean_HR,mean_E(eye)/V(verbal)/M(motor),mean_SpO2,mean_FIO2,"max_Glucose(BST,POCT)","max_Sodium(Na),응급","max_BUN,응급","max_Chloride(Cl),응급","max_Potassium(K),응급","max_GFR by MDRD,응급","max_Creatinine,응급","max_Glucose,WB",max_Standard bicarbonate,max_O2Hb,max_MetHb,"max_Potassium,WB",max_Chloride,max_Sodium,max_Actual base excess,"max_Chloride,WB",max_Standard base excess,max_pO2,max_HHb,"max_Ionized calcium,WB",max_Oxygen saturation,"max_Sodium,WB",max_pCO2,max_Total Hb,"max_Lactate,WB",max_Potassium,"max_pH(ER,ICU,CCU,OR)",max_HCO3- conc.,max_COHb,max_Total CO2 conc.,"min_Glucose(BST,POCT)","min_Sodium(Na),응급","min_BUN,응급","min_Chloride(Cl),응급","min_Potassium(K),응급","min_GFR by MDRD,응급","min_Creatinine,응급","min_Glucose,WB",min_Standard bicarbonate,min_O2Hb,min_MetHb,"min_Potassium,WB",min_Chloride,min_Sodium,min_Actual base excess,"min_Chloride,WB",min_Standard base excess,min_pO2,min_HHb,"min_Ionized calcium,WB",min_Oxygen saturation,"min_Sodium,WB",min_pCO2,min_Total Hb,"min_Lactate,WB",min_Potassium,"min_pH(ER,ICU,CCU,OR)",min_HCO3- conc.,min_COHb,min_Total CO2 conc.,"mean_Glucose(BST,POCT)","mean_Sodium(Na),응급","mean_BUN,응급","mean_Chloride(Cl),응급","mean_Potassium(K),응급","mean_GFR by MDRD,응급","mean_Creatinine,응급","mean_Glucose,WB",mean_Standard bicarbonate,mean_O2Hb,mean_MetHb,"mean_Potassium,WB",mean_Chloride,mean_Sodium,mean_Actual base excess,"mean_Chloride,WB",mean_Standard base excess,mean_pO2,mean_HHb,"mean_Ionized calcium,WB",mean_Oxygen saturation,"mean_Sodium,WB",mean_pCO2,mean_Total Hb,"mean_Lactate,WB",mean_Potassium,"mean_pH(ER,ICU,CCU,OR)",mean_HCO3- conc.,mean_COHb,mean_Total CO2 conc.,inpute_max_RR,inpute_max_BT,inpute_max_DBP,inpute_max_mean BP,inpute_max_SBP,inpute_max_HR,inpute_max_E(eye)/V(verbal)/M(motor),inpute_max_SpO2,inpute_max_FIO2,inpute_min_RR,inpute_min_BT,inpute_min_DBP,inpute_min_mean BP,inpute_min_SBP,inpute_min_HR,inpute_min_E(eye)/V(verbal)/M(motor),inpute_min_SpO2,inpute_min_FIO2,inpute_mean_RR,inpute_mean_BT,inpute_mean_DBP,inpute_mean_mean BP,inpute_mean_SBP,inpute_mean_HR,inpute_mean_E(eye)/V(verbal)/M(motor),inpute_mean_SpO2,inpute_mean_FIO2,"inpute_max_Glucose(BST,POCT)","inpute_max_Sodium(Na),응급","inpute_max_BUN,응급","inpute_max_Chloride(Cl),응급","inpute_max_Potassium(K),응급","inpute_max_GFR by MDRD,응급","inpute_max_Creatinine,응급","inpute_max_Glucose,WB",inpute_max_Standard bicarbonate,inpute_max_O2Hb,inpute_max_MetHb,"inpute_max_Potassium,WB",inpute_max_Chloride,inpute_max_Sodium,inpute_max_Actual base excess,"inpute_max_Chloride,WB",inpute_max_Standard base excess,inpute_max_pO2,inpute_max_HHb,"inpute_max_Ionized calcium,WB",inpute_max_Oxygen saturation,"inpute_max_Sodium,WB",inpute_max_pCO2,inpute_max_Total Hb,"inpute_max_Lactate,WB",inpute_max_Potassium,"inpute_max_pH(ER,ICU,CCU,OR)",inpute_max_HCO3- conc.,inpute_max_COHb,inpute_max_Total CO2 conc.,"inpute_min_Glucose(BST,POCT)","inpute_min_Sodium(Na),응급","inpute_min_BUN,응급","inpute_min_Chloride(Cl),응급","inpute_min_Potassium(K),응급","inpute_min_GFR by MDRD,응급","inpute_min_Creatinine,응급","inpute_min_Glucose,WB",inpute_min_Standard bicarbonate,inpute_min_O2Hb,inpute_min_MetHb,"inpute_min_Potassium,WB",inpute_min_Chloride,inpute_min_Sodium,inpute_min_Actual base excess,"inpute_min_Chloride,WB",inpute_min_Standard base excess,inpute_min_pO2,inpute_min_HHb,"inpute_min_Ionized calcium,WB",inpute_min_Oxygen saturation,"inpute_min_Sodium,WB",inpute_min_pCO2,inpute_min_Total Hb,"inpute_min_Lactate,WB",inpute_min_Potassium,"inpute_min_pH(ER,ICU,CCU,OR)",inpute_min_HCO3- conc.,inpute_min_COHb,inpute_min_Total CO2 conc.,"inpute_mean_Glucose(BST,POCT)","inpute_mean_Sodium(Na),응급","inpute_mean_BUN,응급","inpute_mean_Chloride(Cl),응급","inpute_mean_Potassium(K),응급","inpute_mean_GFR by MDRD,응급","inpute_mean

In [22]:
max_seq_len = 59
n_col = val_test_x_padded.shape[1]
new_dfs = []
for df in dfs:
    tmp = df.values.reshape(-1, max_seq_len, n_col)
    mask_padded = tmp[:, :, 1] == 0
    len_seqs = 59-np.sum(mask_padded, axis=1)
    seqs = []

    for idx, ls in tqdm(enumerate(len_seqs)):
        if ls == 1:
            _seq = tmp[idx].copy()
            seqs.append(_seq.copy())
            continue
        
        hadm_id = tmp[idx, 0, 0]
        result = tmp[idx, 0, -1]
        
        for i in range(1, ls):
            _seq = tmp[idx].copy()
            _seq[i:, :] = [[hadm_id, '0']+np.repeat(0, n_col-3).tolist()+[result] for _ in range(max_seq_len-i)]
            seqs.append(_seq.copy())
        
        _seq = tmp[idx].copy()
        seqs.append(_seq.copy())

    seqs = np.array(seqs)
    seqs = seqs.reshape(-1, n_col)

    new_dfs.append(pd.DataFrame(seqs, columns=df.columns))

34it [00:00, 98.53it/s] 
36it [00:00, 86.79it/s]


In [23]:
train_x_padded

,STAY_ID,time,max_RR,max_BT,max_DBP,max_mean BP,max_SBP,max_HR,max_E(eye)/V(verbal)/M(motor),max_SpO2,max_FIO2,min_RR,min_BT,min_DBP,min_mean BP,min_SBP,min_HR,min_E(eye)/V(verbal)/M(motor),min_SpO2,min_FIO2,mean_RR,mean_BT,mean_DBP,mean_mean BP,mean_SBP,mean_HR,mean_E(eye)/V(verbal)/M(motor),mean_SpO2,mean_FIO2,"max_Glucose(BST,POCT)","max_Sodium(Na),응급","max_BUN,응급","max_Chloride(Cl),응급","max_Potassium(K),응급","max_GFR by MDRD,응급","max_Creatinine,응급","max_Glucose,WB",max_Standard bicarbonate,max_O2Hb,max_MetHb,"max_Potassium,WB",max_Chloride,max_Sodium,max_Actual base excess,"max_Chloride,WB",max_Standard base excess,max_pO2,max_HHb,"max_Ionized calcium,WB",max_Oxygen saturation,"max_Sodium,WB",max_pCO2,max_Total Hb,"max_Lactate,WB",max_Potassium,"max_pH(ER,ICU,CCU,OR)",max_HCO3- conc.,max_COHb,max_Total CO2 conc.,"min_Glucose(BST,POCT)","min_Sodium(Na),응급","min_BUN,응급","min_Chloride(Cl),응급","min_Potassium(K),응급","min_GFR by MDRD,응급","min_Creatinine,응급","min_Glucose,WB",min_Standard bicarbonate,min_O2Hb,min_MetHb,"min_Potassium,WB",min_Chloride,min_Sodium,min_Actual base excess,"min_Chloride,WB",min_Standard base excess,min_pO2,min_HHb,"min_Ionized calcium,WB",min_Oxygen saturation,"min_Sodium,WB",min_pCO2,min_Total Hb,"min_Lactate,WB",min_Potassium,"min_pH(ER,ICU,CCU,OR)",min_HCO3- conc.,min_COHb,min_Total CO2 conc.,"mean_Glucose(BST,POCT)","mean_Sodium(Na),응급","mean_BUN,응급","mean_Chloride(Cl),응급","mean_Potassium(K),응급","mean_GFR by MDRD,응급","mean_Creatinine,응급","mean_Glucose,WB",mean_Standard bicarbonate,mean_O2Hb,mean_MetHb,"mean_Potassium,WB",mean_Chloride,mean_Sodium,mean_Actual base excess,"mean_Chloride,WB",mean_Standard base excess,mean_pO2,mean_HHb,"mean_Ionized calcium,WB",mean_Oxygen saturation,"mean_Sodium,WB",mean_pCO2,mean_Total Hb,"mean_Lactate,WB",mean_Potassium,"mean_pH(ER,ICU,CCU,OR)",mean_HCO3- conc.,mean_COHb,mean_Total CO2 conc.,inpute_max_RR,inpute_max_BT,inpute_max_DBP,inpute_max_mean BP,inpute_max_SBP,inpute_max_HR,inpute_max_E(eye)/V(verbal)/M(motor),inpute_max_SpO2,inpute_max_FIO2,inpute_min_RR,inpute_min_BT,inpute_min_DBP,inpute_min_mean BP,inpute_min_SBP,inpute_min_HR,inpute_min_E(eye)/V(verbal)/M(motor),inpute_min_SpO2,inpute_min_FIO2,inpute_mean_RR,inpute_mean_BT,inpute_mean_DBP,inpute_mean_mean BP,inpute_mean_SBP,inpute_mean_HR,inpute_mean_E(eye)/V(verbal)/M(motor),inpute_mean_SpO2,inpute_mean_FIO2,"inpute_max_Glucose(BST,POCT)","inpute_max_Sodium(Na),응급","inpute_max_BUN,응급","inpute_max_Chloride(Cl),응급","inpute_max_Potassium(K),응급","inpute_max_GFR by MDRD,응급","inpute_max_Creatinine,응급","inpute_max_Glucose,WB",inpute_max_Standard bicarbonate,inpute_max_O2Hb,inpute_max_MetHb,"inpute_max_Potassium,WB",inpute_max_Chloride,inpute_max_Sodium,inpute_max_Actual base excess,"inpute_max_Chloride,WB",inpute_max_Standard base excess,inpute_max_pO2,inpute_max_HHb,"inpute_max_Ionized calcium,WB",inpute_max_Oxygen saturation,"inpute_max_Sodium,WB",inpute_max_pCO2,inpute_max_Total Hb,"inpute_max_Lactate,WB",inpute_max_Potassium,"inpute_max_pH(ER,ICU,CCU,OR)",inpute_max_HCO3- conc.,inpute_max_COHb,inpute_max_Total CO2 conc.,"inpute_min_Glucose(BST,POCT)","inpute_min_Sodium(Na),응급","inpute_min_BUN,응급","inpute_min_Chloride(Cl),응급","inpute_min_Potassium(K),응급","inpute_min_GFR by MDRD,응급","inpute_min_Creatinine,응급","inpute_min_Glucose,WB",inpute_min_Standard bicarbonate,inpute_min_O2Hb,inpute_min_MetHb,"inpute_min_Potassium,WB",inpute_min_Chloride,inpute_min_Sodium,inpute_min_Actual base excess,"inpute_min_Chloride,WB",inpute_min_Standard base excess,inpute_min_pO2,inpute_min_HHb,"inpute_min_Ionized calcium,WB",inpute_min_Oxygen saturation,"inpute_min_Sodium,WB",inpute_min_pCO2,inpute_min_Total Hb,"inpute_min_Lactate,WB",inpute_min_Potassium,"inpute_min_pH(ER,ICU,CCU,OR)",inpute_min_HCO3- conc.,inpute_min_COHb,inpute_min_Total CO2 conc.,"inpute_mean_Glucose(BST,POCT)","inpute_mean_Sodium(Na),응급","inpute_mean_BUN,응급","inpute_mean_Chloride(Cl),응급","inpute_mean_Potassium(K),응급","inpute_mean_GFR by MDRD,응급","inpute_mean

In [24]:
df_raw_dict_12hour = {
    "train_x": train_x_padded,
    "val_x": val_x_padded,
    "val_test_x": new_dfs[0],
    "test_x": new_dfs[1],
}

In [25]:
import pickle

with open(f'df_raw_dict_12hour.pkl', 'wb') as f:
    pickle.dump(df_raw_dict_12hour, f)